# Evaluate a MedCATtrainer project export

In [ ]:
from mct_analysis import MedcatTrainer_export

In [ ]:
mct_export = '../../data/medcattrainer_export/'+''  # mct_export .json here
mct_model = ""
mct = MedcatTrainer_export([mct_export+''], model_pack_path= mct_model)

In [ ]:
# annotations
anns_df = mct.annotation_df()
anns_df

In [ ]:
# Meta_annotation summary
for col in anns_df.loc[:,'acc':].iloc[:,1:]:
    print(anns_df[col].value_counts())

In [ ]:
# meta_annotation summary of combinations
for k,v in anns_df.loc[:,'acc':].iloc[:,1:].value_counts().iteritems():
    print(k,v)

In [ ]:
# projects
anns_df['project'].unique()

In [ ]:
# documents
anns_df['document_name'].unique()

# Annotation Summary

In [ ]:
mct.concept_summary()

# Annotator stats

In [ ]:
# User Stats
mct.user_stats()

In [ ]:
mct.plot_user_stats()

# Meta annotation summary

In [ ]:
anns_df